# VerveStacks Wind Interactive Map - Enhanced Version

## Overview
This notebook creates professional, interactive wind resource maps for both offshore and onshore wind energy using high-resolution Atlite capacity factor data. The maps provide comprehensive analysis of wind potential, economic viability, and resource quality for any ISO country.

## Features

### 🌬️ **Dual Wind Type Support**
- **Offshore Wind**: Ocean-based wind resources with blue color scheme
- **Onshore Wind**: Land-based wind resources with green color scheme

### 🎨 **Enhanced Visualization**
- **6-Tier Color Scheme**: Capacity factor ranges from 0.30 to 0.50+
- **4-Tier Marker Sizing**: Visual representation of capacity potential
- **Professional Styling**: Clean borders, enhanced popups, and CSS styling

### 📊 **Comprehensive Analysis**
- **Resource Quality**: Capacity factors and zone scores
- **Energy Potential**: Installed capacity, annual generation, and density
- **Economics**: LCOE and suitable area analysis
- **Location Data**: Precise coordinates and wind type identification

### 🗺️ **Interactive Features**
- **Multiple Tile Layers**: OpenStreetMap, CartoDB Positron, CartoDB Dark, Terrain
- **Layer Control**: Switch between different base maps
- **Measurement Tools**: Distance and area measurement capabilities
- **Fullscreen Mode**: Enhanced viewing experience
- **Enhanced Popups**: Detailed zone information with professional styling

## Data Sources

### **Primary Data**
- **Offshore Wind**: `REZoning_WindOffshore_atlite_cf.csv`
- **Onshore Wind**: `REZoning_WindOnshore_atlite_cf.csv`
- **Costs**: `REZoning_costs_per_kw.csv`

### **Data Quality**
- **Atlite Capacity Factors**: High-resolution ERA5 weather data
- **Technology-Specific Modeling**: Accurate wind resource assessment
- **Global Coverage**: 136+ countries for offshore, 170+ countries for onshore

## Usage

### **Basic Usage**
1. **Load Data**: Run cells 1-2 to load wind data and costs
2. **Select ISO**: Change `sample_iso` variable in cell 3 to your target country
3. **Generate Maps**: Run cell 4 to create both offshore and onshore maps
4. **View Results**: Maps are saved as HTML files in the `output/` directory

### **Customization**
- **Analysis Level**: Change `analysis_level` parameter ('basic', 'intermediate', 'comprehensive')
- **Minimum Capacity**: Adjust `min_capacity_mw` threshold
- **Color Schemes**: Modify color ranges in the `get_enhanced_wind_capacity_factor_color` function

## Output Files
- `enhanced_offshore_wind_zones_map_{ISO}_atlite.html`
- `enhanced_onshore_wind_zones_map_{ISO}_atlite.html`

## Technical Details

### **Color Schemes**
- **Offshore**: Blue gradient (Deep Blue → Powder Blue)
- **Onshore**: Green gradient (Dark Green → Khaki)

### **Capacity Factor Ranges**
- **Excellent**: ≥0.50
- **High**: 0.45-0.50
- **Good**: 0.40-0.45
- **Fair**: 0.35-0.40
- **Poor**: 0.30-0.35
- **Very Poor**: <0.30

### **Performance Optimization**
- **Sampling**: Maximum 3000 zones for comprehensive analysis
- **Efficient Rendering**: Optimized for large datasets
- **Memory Management**: Smart data processing and visualization

## Requirements
- Python 3.6+
- pandas, numpy, folium
- geopandas (auto-installed if missing)

---
**Created by VerveStacks** | **Enhanced Wind Resource Analysis** | **Atlite ERA5 Weather Data**


In [21]:
# VerveStacks Wind Interactive Map - Enhanced Version (Offshore & Onshore)
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up paths
data_path = Path('../data/REZoning')
offshore_file = data_path / 'REZoning_WindOffshore_atlite_cf.csv'
onshore_file = data_path / 'REZoning_WindOnshore_atlite_cf.csv'
costs_file = data_path / 'REZoning_costs_per_kw.csv'

print("VerveStacks Wind Interactive Map - Enhanced Version")
print("=" * 60)


VerveStacks Wind Interactive Map - Enhanced Version


In [22]:
# Load wind data
print("Loading wind data...")
offshore_data = pd.read_csv(offshore_file)
onshore_data = pd.read_csv(onshore_file)
costs_data = pd.read_csv(costs_file)

print(f"Offshore wind data shape: {offshore_data.shape}")
print(f"Onshore wind data shape: {onshore_data.shape}")
print(f"Available offshore ISOs: {len(offshore_data['ISO'].dropna().unique())} countries")
print(f"Available onshore ISOs: {len(onshore_data['ISO'].dropna().unique())} countries")

# Show sample ISOs for both
offshore_isos = sorted(offshore_data['ISO'].dropna().unique())
onshore_isos = sorted(onshore_data['ISO'].dropna().unique())
print(f"Sample offshore ISOs: {offshore_isos[:10]}")
print(f"Sample onshore ISOs: {onshore_isos[:10]}")


Loading wind data...
Offshore wind data shape: (15782, 14)
Onshore wind data shape: (47285, 13)
Available offshore ISOs: 136 countries
Available onshore ISOs: 170 countries
Sample offshore ISOs: ['AGO', 'ALB', 'ARE', 'ARG', 'ATG', 'AUS', 'AZE', 'BEN', 'BGD', 'BGR']
Sample onshore ISOs: ['AFG', 'AGO', 'ALB', 'ARE', 'ARG', 'ARM', 'ATG', 'AUS', 'AUT', 'AZE']


In [23]:
# Core function to create wind grid data (offshore or onshore)
def create_wind_grid_for_iso(iso_code, wind_df, costs_df, wind_type='onshore', min_capacity_mw=1.0):
    """Create wind grid analysis for a specific ISO code (offshore or onshore)."""
    
    # Filter data for the specific ISO
    iso_wind = wind_df[wind_df['ISO'] == iso_code].copy()
    
    if iso_wind.empty:
        print(f"No {wind_type} wind data found for ISO: {iso_code}")
        return None
    
    # Get cost data for this ISO
    iso_costs = costs_df[costs_df['iso'] == iso_code]
    
    # Clean and process the data - wind data uses 'lng' instead of 'long'
    iso_wind = iso_wind.dropna(subset=['lat', 'lng', 'Capacity Factor'])
    iso_wind = iso_wind[iso_wind['Installed Capacity Potential (MW)'] >= min_capacity_mw]
    
    # Rename 'lng' to 'long' for consistency with solar data
    iso_wind = iso_wind.rename(columns={'lng': 'long'})
    
    # Calculate additional metrics
    iso_wind['Total_Generation_GWh'] = (
        iso_wind['Installed Capacity Potential (MW)'] * 
        iso_wind['Capacity Factor'] * 8760 / 1000
    )
    
    # Create grid statistics
    grid_stats = {
        'iso': iso_code,
        'wind_type': wind_type,
        'total_cells': len(iso_wind),
        'total_capacity_mw': iso_wind['Installed Capacity Potential (MW)'].sum(),
        'total_generation_gwh': iso_wind['Total_Generation_GWh'].sum(),
        'avg_capacity_factor': iso_wind['Capacity Factor'].mean(),
        'avg_lcoe': iso_wind['LCOE (USD/MWh)'].mean(),
        'total_suitable_area_km2': iso_wind['Suitable Area (km²)'].sum(),
        'cost_data_available': not iso_costs.empty,
        'investment_cost_usd_kw': iso_costs['invcost'].iloc[0] if not iso_costs.empty else None,
        'fixed_om_usd_kw': iso_costs['fixom'].iloc[0] if not iso_costs.empty else None,
        'data_source': f'Atlite {wind_type.title()} Wind (High-resolution ERA5 weather data)',
        'capacity_factor_quality': 'High-resolution, technology-specific modeling'
    }
    
    return {
        'grid_data': iso_wind,
        'statistics': grid_stats,
        'costs': iso_costs
    }

# Create grids for both offshore and onshore (example with USA)
sample_iso = 'USA'
print(f"Creating wind grids for ISO: {sample_iso}")

# Offshore wind
offshore_result = create_wind_grid_for_iso(sample_iso, offshore_data, costs_data, 'offshore')
if offshore_result:
    stats = offshore_result['statistics']
    print(f"Offshore Wind Statistics for {sample_iso}:")
    print(f"Total grid cells: {stats['total_cells']:,}")
    print(f"Total capacity: {stats['total_capacity_mw']:,.1f} MW")
    print(f"Average capacity factor: {stats['avg_capacity_factor']:.3f}")

# Onshore wind
onshore_result = create_wind_grid_for_iso(sample_iso, onshore_data, costs_data, 'onshore')
if onshore_result:
    stats = onshore_result['statistics']
    print(f"Onshore Wind Statistics for {sample_iso}:")
    print(f"Total grid cells: {stats['total_cells']:,}")
    print(f"Total capacity: {stats['total_capacity_mw']:,.1f} MW")
    print(f"Average capacity factor: {stats['avg_capacity_factor']:.3f}")


Creating wind grids for ISO: USA
Offshore Wind Statistics for USA:
Total grid cells: 241
Total capacity: 550,620.0 MW
Average capacity factor: 0.208
Onshore Wind Statistics for USA:
Total grid cells: 3,909
Total capacity: 6,038,180.2 MW
Average capacity factor: 0.258


In [ ]:
# ENHANCED WIND INTERACTIVE MAP WITH IMPROVED COLOR SCHEME
import folium
from folium import plugins
import numpy as np
import pandas as pd

def create_enhanced_wind_interactive_map(grid_result, analysis_level='comprehensive'):
    """
    Create an enhanced interactive wind map with improved color scheme for better visibility.
    
    Parameters:
    -----------
    grid_result : dict
        Result from create_wind_grid_for_iso function
    analysis_level : str
        Level of analysis ('basic', 'intermediate', 'comprehensive')
    """
    
    if not grid_result:
        print("No grid data to visualize")
        return
    
    grid_data = grid_result['grid_data']
    stats = grid_result['statistics']
    wind_type = stats['wind_type']
    
    # Calculate center point for map
    center_lat = grid_data['lat'].mean()
    center_lon = grid_data['long'].mean()
    
    # Create the enhanced map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=6,
        tiles='OpenStreetMap'
    )
    
    # Add multiple tile layers for better analysis (with proper attributions)
    folium.TileLayer('CartoDB positron', name='CartoDB Positron').add_to(m)
    folium.TileLayer('CartoDB dark_matter', name='CartoDB Dark').add_to(m)
    folium.TileLayer(
        tiles='https://stamen-tiles-{s}.a.ssl.fastly.net/terrain/{z}/{x}/{y}{r}.png',
        attr='Map tiles by <a href="http://stamen.com">Stamen Design</a>, under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>. Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, under <a href="http://www.openstreetmap.org/copyright">ODbL</a>.',
        name='Terrain',
        overlay=False,
        control=True
    ).add_to(m)
    
    # WIND-SPECIFIC color scheme with better contrast and visibility
    def get_enhanced_wind_capacity_factor_color(cf, wind_type):
        if wind_type == 'offshore':
            # Offshore wind colors (blues/teals for ocean theme)
            if cf >= 0.50:
                return '#0066CC'  # Deep Blue - Excellent
            elif cf >= 0.45:
                return '#0080FF'  # Bright Blue - High
            elif cf >= 0.40:
                return '#00BFFF'  # Sky Blue - Good
            elif cf >= 0.35:
                return '#40E0D0'  # Turquoise - Fair
            elif cf >= 0.30:
                return '#87CEEB'  # Sky Blue - Poor
            else:
                return '#B0E0E6'  # Powder Blue - Very Poor
        else:
            # Onshore wind colors (greens for land theme)
            if cf >= 0.50:
                return '#006400'  # Dark Green - Excellent
            elif cf >= 0.45:
                return '#228B22'  # Forest Green - High
            elif cf >= 0.40:
                return '#32CD32'  # Lime Green - Good
            elif cf >= 0.35:
                return '#9ACD32'  # Yellow Green - Fair
            elif cf >= 0.30:
                return '#ADFF2F'  # Green Yellow - Poor
            else:
                return '#F0E68C'  # Khaki - Very Poor
    
    # Enhanced marker size calculation
    def get_enhanced_marker_size(capacity_mw):
        if capacity_mw >= 5000:
            return 22
        elif capacity_mw >= 2000:
            return 18
        elif capacity_mw >= 1000:
            return 14
        elif capacity_mw >= 500:
            return 12
        elif capacity_mw >= 100:
            return 10
        elif capacity_mw >= 50:
            return 8
        else:
            return 6
    
    # Sample zones for performance (show max 3000 zones for comprehensive analysis)
    max_zones = 3000 if analysis_level == 'comprehensive' else 2000
    map_sample = grid_data.sample(n=min(max_zones, len(grid_data)), random_state=42).copy()
    
    print(f"Creating enhanced {wind_type} wind map with {len(map_sample)} zones ({analysis_level} analysis)...")
    
    # Add wind zones with enhanced analysis
    for idx, zone in map_sample.iterrows():
        # Get zone details
        capacity_factor = zone['Capacity Factor']
        capacity_mw = zone['Installed Capacity Potential (MW)']
        lcoe = zone['LCOE (USD/MWh)']
        generation_gwh = zone['Total_Generation_GWh']
        area_km2 = zone['Suitable Area (km²)']
        zone_score = zone['Zone Score']
        
        # Calculate generation density safely
        generation_density = generation_gwh/area_km2 if area_km2 > 0 else float('nan')
        density_text = f"{generation_density:.2f}" if not np.isnan(generation_density) else "N/A"
        
        # Enhanced popup content with wind-specific analysis
        wind_icon = "🌊" if wind_type == 'offshore' else "🌬️"
        wind_title = f"{wind_type.title()} Wind Zone"
        
        popup_content = f"""
        <div style="font-family: Arial, sans-serif; width: 320px;">
            <h4 style="margin: 0; color: #2C3E50; background: #ECF0F1; padding: 8px; border-radius: 5px;">
                {wind_icon} {wind_title}: {zone['grid_cell']}
            </h4>
            <div style="padding: 10px;">
                <div style="background: #F8F9FA; padding: 8px; border-radius: 3px; margin: 5px 0;">
                    <h5 style="margin: 0; color: #34495E;">📊 Resource Quality</h5>
                    <p style="margin: 2px 0;"><b>Capacity Factor:</b> {capacity_factor:.3f}</p>
                    <p style="margin: 2px 0;"><b>Zone Score:</b> {zone_score:.3f}</p>
                </div>
                
                <div style="background: #E8F5E8; padding: 8px; border-radius: 3px; margin: 5px 0;">
                    <h5 style="margin: 0; color: #27AE60;">⚡ Energy Potential</h5>
                    <p style="margin: 2px 0;"><b>Installed Capacity:</b> {capacity_mw:.1f} MW</p>
                    <p style="margin: 2px 0;"><b>Annual Generation:</b> {generation_gwh:.1f} GWh</p>
                    <p style="margin: 2px 0;"><b>Generation Density:</b> {density_text} GWh/km²</p>
                </div>
                
                <div style="background: #FFF3CD; padding: 8px; border-radius: 3px; margin: 5px 0;">
                    <h5 style="margin: 0; color: #856404;">💰 Economics</h5>
                    <p style="margin: 2px 0;"><b>LCOE:</b> ${lcoe:.2f}/MWh</p>
                    <p style="margin: 2px 0;"><b>Suitable Area:</b> {area_km2:.1f} km²</p>
                </div>
                
                <div style="background: #D1ECF1; padding: 8px; border-radius: 3px; margin: 5px 0;">
                    <h5 style="margin: 0; color: #0C5460;">📍 Location</h5>
                    <p style="margin: 2px 0;"><b>Coordinates:</b> {zone['lat']:.3f}, {zone['long']:.3f}</p>
                    <p style="margin: 2px 0;"><b>Type:</b> {wind_type.title()} Wind</p>
                </div>
            </div>
        </div>
        """
        
        # Add marker with enhanced styling and CSS class
        folium.CircleMarker(
            location=[zone['lat'], zone['long']],
            radius=get_enhanced_marker_size(capacity_mw),
            popup=folium.Popup(popup_content, max_width=350),
            tooltip=f"{wind_icon} Zone {zone['grid_cell']} (CF: {capacity_factor:.3f}, {capacity_mw:.1f} MW)",
            color='white',
            weight=3,  # Thicker white border for better contrast
            fillColor=get_enhanced_wind_capacity_factor_color(capacity_factor, wind_type),
            fillOpacity=0.85,  # Slightly more opaque for better visibility
            className='wind-circle'  # Apply CSS class for stroke-width styling
        ).add_to(m)
    
    # Enhanced legend with wind-specific color scheme and CSS styling
    legend_html = f'''
    <style>
    .wind-circle {{
        stroke-width: 1px !important;
        stroke: white !important;
    }}
    </style>
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 300px; height: 320px; 
                background-color: white; border:3px solid #34495E; z-index:9999; 
                font-size:13px; padding: 15px; border-radius: 10px; box-shadow: 0 6px 12px rgba(0,0,0,0.15);">
    <h4 style="margin: 0 0 15px 0; color: #2C3E50; text-align: center; font-weight: bold;">{wind_title} Analysis</h4>
    
    <div style="margin-bottom: 12px;">
        <h5 style="margin: 0 0 8px 0; color: #34495E; font-weight: bold;">📊 Resource Quality (CF)</h5>'''
    
    # Add wind-specific color legend
    if wind_type == 'offshore':
        legend_html += f'''
        <p style="margin: 3px 0;"><span style="color:#0066CC; font-weight: bold;">●</span> Excellent (≥0.50)</p>
        <p style="margin: 3px 0;"><span style="color:#0080FF; font-weight: bold;">●</span> High (0.45-0.50)</p>
        <p style="margin: 3px 0;"><span style="color:#00BFFF; font-weight: bold;">●</span> Good (0.40-0.45)</p>
        <p style="margin: 3px 0;"><span style="color:#40E0D0; font-weight: bold;">●</span> Fair (0.35-0.40)</p>
        <p style="margin: 3px 0;"><span style="color:#87CEEB; font-weight: bold;">●</span> Poor (0.30-0.35)</p>
        <p style="margin: 3px 0;"><span style="color:#B0E0E6; font-weight: bold;">●</span> Very Poor (<0.30)</p>'''
    else:
        legend_html += f'''
        <p style="margin: 3px 0;"><span style="color:#006400; font-weight: bold;">●</span> Excellent (≥0.50)</p>
        <p style="margin: 3px 0;"><span style="color:#228B22; font-weight: bold;">●</span> High (0.45-0.50)</p>
        <p style="margin: 3px 0;"><span style="color:#32CD32; font-weight: bold;">●</span> Good (0.40-0.45)</p>
        <p style="margin: 3px 0;"><span style="color:#9ACD32; font-weight: bold;">●</span> Fair (0.35-0.40)</p>
        <p style="margin: 3px 0;"><span style="color:#ADFF2F; font-weight: bold;">●</span> Poor (0.30-0.35)</p>
        <p style="margin: 3px 0;"><span style="color:#F0E68C; font-weight: bold;">●</span> Very Poor (<0.30)</p>'''
    
    legend_html += f'''
    </div>
    
    <div style="margin-bottom: 12px;">
        <h5 style="margin: 0 0 8px 0; color: #34495E; font-weight: bold;">⚡ Zone Size (MW)</h5>
        <p style="margin: 3px 0;">● Small: <100 MW</p>
        <p style="margin: 3px 0;">●● Medium: 100-1000 MW</p>
        <p style="margin: 3px 0;">●●● Large: 1000-5000 MW</p>
        <p style="margin: 3px 0;">●●●● Very Large: >5000 MW</p>
    </div>
    
    <div style="background: #F8F9FA; padding: 10px; border-radius: 5px; border-left: 4px solid #3498DB;">
        <p style="margin: 2px 0; font-size: 11px; color: #7F8C8D; font-style: italic;"><b>Atlite ERA5 Weather Data</b></p>
        <p style="margin: 2px 0; font-size: 11px; color: #7F8C8D; font-style: italic;">Analysis Level: {analysis_level.title()}</p>
        <p style="margin: 2px 0; font-size: 11px; color: #7F8C8D; font-style: italic;">Wind Type: {wind_type.title()}</p>
    </div>
    </div>
    '''
    
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    # Add measurement tools
    from folium.plugins import MeasureControl
    m.add_child(MeasureControl())
    
    # Add fullscreen button
    from folium.plugins import Fullscreen
    Fullscreen().add_to(m)
    
    return m

# Create enhanced interactive maps for both offshore and onshore wind
if offshore_result:
    print("Creating enhanced offshore wind interactive map...")
    offshore_map = create_enhanced_wind_interactive_map(offshore_result, 'comprehensive')
    
    # Save the offshore map
    offshore_map_file = f'output/enhanced_offshore_wind_zones_map_{stats["iso"]}_atlite.html'
    offshore_map.save(offshore_map_file)
    
    print(f"✅ Enhanced offshore wind map created!")
    print(f"✅ Map saved to: {offshore_map_file}")
    
    # Display the offshore map
    offshore_map

if onshore_result:
    print("Creating enhanced onshore wind interactive map...")
    onshore_map = create_enhanced_wind_interactive_map(onshore_result, 'comprehensive')
    
    # Save the onshore map
    onshore_map_file = f'output/enhanced_onshore_wind_zones_map_{stats["iso"]}_atlite.html'
    onshore_map.save(onshore_map_file)
    
    print(f"✅ Enhanced onshore wind map created!")
    print(f"✅ Map saved to: {onshore_map_file}")
    
    # Display the onshore map
    onshore_map


Creating enhanced offshore wind interactive map...
Creating enhanced offshore wind map with 241 zones (comprehensive analysis)...
✅ Enhanced offshore wind map created!
✅ Map saved to: output/enhanced_offshore_wind_zones_map_USA_atlite.html
Creating enhanced onshore wind interactive map...
Creating enhanced onshore wind map with 3000 zones (comprehensive analysis)...
✅ Enhanced onshore wind map created!
✅ Map saved to: output/enhanced_onshore_wind_zones_map_USA_atlite.html


: 